#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%pip install ffmpeg
%pip install grakel

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import networkx as nx
import pickle
import cv2
import math
import json
import os
from copy import copy
from copy import deepcopy
from google.colab.patches import cv2_imshow
from IPython.display import HTML

from grakel import GraphKernel
from collections import defaultdict
from collections import Counter
from sklearn.cluster import Birch
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics

import scipy.sparse as sp
import scipy.sparse.linalg as splinalg

from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')

rootdir = "/content/drive/MyDrive/XAMI-MIMIC"
%matplotlib inline


# show all rows from dataframe
pd.set_option('display.max_rows', 100)
# show all columns from dataframe
pd.set_option('display.max_columns', 100)
from IPython.display import Image, clear_output
clear_output()

In [ ]:
with open("/content/drive/MyDrive/Colab-Variables/reflacx_gaze.pkl",'rb') as file:
  _reflacx = pickle.load(file)
  #_reflacx is in the form of {}

#Sparse Functions

In [ ]:
#@title Sequential Graphs
def sequential_graph(graph,nb_clusters,points,centroid_points,img_h,img_w,img,type_xray,patient,timestamped,ellipses_points):

  cluster_order = np.full(nb_clusters, -1)
  for index, value in np.ndenumerate(points):
    if value < nb_clusters and cluster_order[value] == -1:
      cluster_order[value] = index[0]
  sorted_indices = (np.argsort(cluster_order)).tolist()
  ordered_clusters = [(sorted_indices[i], sorted_indices[i+1]) for i in range(len(sorted_indices)-1)]

  my_graph = deepcopy(graph)
  my_graph.add_edges_from(ordered_clusters)

  drawPlotSave(ellipses_points,img_h,img_w,img,my_graph,timestamped,type_xray,'Sequence',patient,nb_clusters)

In [ ]:
#@title NN Graphs
def NNSparcification(graph,k,centroid_points,nb_clusters,img_h,img_w,img,type_xray,patient,timestamped,ellipses_points):
  ax = plt.axes()
  ax.set_ylim([0,img_h])
  ax.set_xlim([0, img_w])
  ax.invert_yaxis()
  ax.imshow(img, cmap="gray")

  my_graph = deepcopy(graph)

  # Use NearestNeighbor to find which nodes will connect to which
  k = len(centroid_points) if len(centroid_points) < k else k
  nn = NearestNeighbors(n_neighbors=k).fit(centroid_points)
  distances, indices = nn.kneighbors(centroid_points)

  for i in range(len(centroid_points)):
    for j in indices[i][1:]:
      my_graph.add_edge(i,j)

  drawPlotSave(ellipses_points,img_h,img_w,img,my_graph,timestamped,type_xray,'NN',patient,nb_clusters)


In [ ]:
#@title Spectral Graphs
def SpectralSparsification(model,pred,gaze,points,ellipses_points,img_height,img_width,img,type_xray,patient,timestamped):

  # create an directed adjacency matrix first
  num_clusters = len(model.subcluster_labels_)
  directed_adjacency =  np.zeros((num_clusters, num_clusters)).astype(int)

  # 1. all the vertices pertaining to each cluster are removed except the centroid of the cluster
  # 2. all the edges that were connecting vertices from different clusters now connect the corresponding centroids
  # 3. all the edges that were connecting vertices inside each cluster are modeled as self loops on the centroid

  gaze['cluster'] = pred.astype(int)
  for i in range(len(gaze)-1):
    u_cluster = gaze.iloc[i]['cluster']
    v_cluster = gaze.iloc[i+1]['cluster']
    directed_adjacency[int(u_cluster), int(v_cluster)] += 1

  # then, transform the adjacency matrix to undirected
  undirected_adjacency = directed_adjacency + directed_adjacency.T
  for i in range(len(undirected_adjacency)):
    undirected_adjacency[i, i] /=2

  # create a graph
  centroid_x = model.subcluster_centers_[:, 0]
  centroid_y = model.subcluster_centers_[:, 1]

  node_counter = dict(Counter(pred))        # number of nodes in a cluster
  todivide = len(points)

  G = nx.Graph()
  # Add nodes to the graph and set their positions
  for (i, x, y) in zip(model.subcluster_labels_, centroid_x, centroid_y):
    N = len(gaze[gaze['cluster'] == i])
    C = undirected_adjacency[i,i] + 1
    G.add_node(i, pos=(x,y), N=N, C=C,size=node_counter[i]/todivide)

  # Add edges to the graph and set their weights
  for i in range(len(undirected_adjacency)):
    for j in range(len(undirected_adjacency[i])):
      if undirected_adjacency[i][j] > 0:
        G.add_edge(i, j, weight=undirected_adjacency[i][j])

  graphs_before_spar[id] = G
  n = G.number_of_nodes()
  m = G.number_of_edges()

  # so first, we get the matrix
  L = nx.laplacian_matrix(G).todense()
  #A = nx.adjacency_matrix(G).todense()

  L_plus = np.linalg.pinv(L)
  effective_resistance = np.zeros(m)
  custom_weights = np.zeros(m)
  for i, (u, v) in enumerate(G.edges()):
    data = G.get_edge_data(u, v)
    e_u = np.zeros(n)
    e_u[u] = 1
    e_v = np.zeros(n)
    e_v[v] = 1

    R_uv = (e_u-e_v).T @ L_plus @ (e_u-e_v)
    effective_resistance[i] = R_uv

    N_u = G.nodes[u]['N']
    N_v = G.nodes[v]['N']
    C_u = G.nodes[u]['C']
    C_v = G.nodes[v]['C']

    # custom_weights[i] = (np.exp(-(N_u**2)*C_u)**(-1)) * (np.exp(-np.power(N_v,2)*C_v)**(-1)) # according to the paper,
    custom_weights[i] = np.e**((-(N_u**2)*C_u).astype(float)**(-1)) * np.e**((-(N_v**2)*C_v).astype(float)**(-1))

  combined_weights = custom_weights * effective_resistance
  p = combined_weights / np.sum(combined_weights)

  pot_k = m - len(list(nx.nodes_with_selfloops(G)))
  k = round(m * 0.6) if round(m * 0.6) <= pot_k else pot_k

  H = nx.Graph()
  H.add_nodes_from(G.nodes(data=True))
  sampled_edges = np.random.choice(m, size=k, replace=False, p=p)
  for i in sampled_edges:
      u, v = list(G.edges())[i]
      H.add_edge(u, v, weight=G.edges()[u,v]['weight'])

  drawPlotSave(ellipses_points,img_height,img_width,img,H,timestamped,type_xray,'Spectral',patient,n)

In [ ]:
#@title Draw Functions and Train BIRCH for All
def elipsedraw(a):
    x = (a[0]+a[2])/2
    y = (a[1]+a[3])/2
    w = a[2]-a[0]
    h = a[3]-a[1]
    return Ellipse(xy=(x, y), width=w, height=h, edgecolor='r', fc='None', lw=0.25)

def drawPlotSave(ellipses_points,img_h,img_w,img,graph,timestamped,type_xray,mode,patient,nb_cluster):
  #"""
  ax = plt.axes()
  ax.set_ylim([0,img_h])
  ax.set_xlim([0, img_w])
  ax.invert_yaxis()
  ax.imshow(img, cmap="gray")

  pos = nx.get_node_attributes(graph, 'pos')
  sizen = nx.get_node_attributes(graph, 'size')
  #nx.draw(graph, pos, node_size=list(sizen),edge_color='purple')
  nx.draw(graph, pos, node_size = 5, edge_color='purple')
  for one_ellipse in ellipses_points:
    tmp = copy(one_ellipse)
    ax.add_patch(tmp)

  plt.savefig(f"/content/drive/MyDrive/{timestamped}_{type_xray}_{patient}_{mode}_clusters-{nb_cluster}")
  #plt.show()
  plt.cla()

In [ ]:
#@title train
def trainBirch(report,gaze,points,path,br,thr):
    # Create the BIRCH model and fit the REFLACX data
    if os.path.exists(path + '/fixations.csv'):
        pass
    else:
        if thr == 90:
            thr = 90.5472
        if(len(points) > 0):
            model = Birch(branching_factor=br, n_clusters=None, threshold = thr)
            model.fit(points)
            pred = model.predict(points)
            SpectralSparsification(report,model,pred,gaze,points,path)
        else:
            df = pd.DataFrame(columns=['x', 'y', 'weight', 'weight_s'])
            df.to_csv(path + '/fixations.csv', index=False)

#Main

In [ ]:
brs = [100,150,200]     # List of branching factor values for BIRCH
thrs = [50]             # List of threshold values for BIRCH

from tqdm.notebook import tqdm


for br in tqdm(brs):
    for thr in tqdm(thrs):
        for key, item in tqdm(_reflacx_speech.items()):
            reflacx_points = np.array([(x,y) for x, y in zip(item['x_position'], item['y_position'])])
            trainBirch(key,item,reflacx_points,dir_path,br,thr)